# CSI-6-ARI Week 4 Tutorial — **COMPLETE ANSWERED VERSION**
## Data Preprocessing

This notebook contains **all code cells answered step-by-step**, including the 4 exercises.

---
### Topics covered
1. StandardScaler (standardisation)
2. Categorical encoding (LabelEncoder & OneHotEncoder)
3. Train / Test split
4. End-to-end preprocessing with MinMaxScaler


# Set up

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

print("Setup complete, seed =", SEED)

Setup complete, seed = 42


## Part 1 — Standardisation with `StandardScaler`

**Key formula:** `z = (x - μ) / σ`
After standardisation every feature column has **mean ≈ 0** and **std ≈ 1**.

This is important for distance-based (KNN, SVM, KMeans) and gradient-based models.

In [6]:
from sklearn.preprocessing import StandardScaler

In [7]:
# --- Option 1: quick one-liner ---
# Feature matrix: 4 samples, 2 features with very different scales
X = np.array([
    [1,  10],
    [3, 100],
    [2,  55],
    [4,  25]
])

# fit_transform: learns mean/std from X and applies z = (x-mean)/std
X_scaled = StandardScaler().fit_transform(X)
print("Standardised X (2 features):")
print(X_scaled)

Standardised X (2 features):
[[-1.34164079 -1.09108945]
 [ 0.4472136   1.52752523]
 [-0.4472136   0.21821789]
 [ 1.34164079 -0.65465367]]


In [ ]:
# --- Option 2: using sklearn.preprocessing directly (3 features) ---
import sklearn

X3 = np.array([
    [1,  10,  0],
    [3, 100, 10],
    [2,  55,  5],
    [4,  25,  2]
])

X3_scaled = sklearn.preprocessing.StandardScaler().fit_transform(X3)
print("Standardised X (3 features):")
print(X3_scaled)
# Notice each column is now on the same scale

---
### Exercise 1 — Check standardisation

**Task:** Create `X2` (shape 5×2) with very different column scales, standardise it, then verify mean≈0 and std≈1.

In [ ]:
# Step 1: Create X2 with two features on very different scales
X2 = np.array([
    [1,    10],
    [2,    50],
    [3,   100],
    [4,    25],
    [5,    60],
])
print("Original X2:")
print(X2)

# Step 2: Standardise using StandardScaler
X2_scaled = StandardScaler().fit_transform(X2)
print("\nScaled X2:")
print(X2_scaled)

# Step 3: Compute per-column mean and std
col_means = X2_scaled.mean(axis=0)
col_stds  = X2_scaled.std(axis=0)

print("\nColumn means (should be ~0):", col_means)
print("Column stds  (should be ~1):", col_stds)

# Explanation:
# StandardScaler standardises each column independently.
# Tiny non-zero means (e.g. 1e-16) are just floating-point rounding errors, not actual bias.

---
## Part 2 — Handling Categorical Variables (Encoding)

Machine learning models need numbers. Two strategies:
- **Label Encoding**: assigns integers (0, 1, 2, …). Implies ordering — OK for ordinal categories.
- **One-Hot Encoding**: creates a binary column per category. No ordering implied — best for nominal categories.

In [ ]:
# Original categorical feature matrix
X_cat = np.array([
    ["Red",   "Petrol", "Sedan"],
    ["Black", "Diesel", "Sedan"],
    ["Blue",  "Diesel", "Hatchback"]
])
print(X_cat)

In [ ]:
from sklearn import preprocessing

# Create a LabelEncoder instance
encoder = preprocessing.LabelEncoder()

# Encode the 'Colour' feature — assigns integers alphabetically
encoded_colours = encoder.fit_transform(["Red", "Black", "Blue"])
print("Label encoded ['Red','Black','Blue']:", encoded_colours)
# Black=0, Blue=1, Red=2  (alphabetical order)

In [ ]:
# Encoding with a new value 'Yellow'
encoded2 = encoder.fit_transform(["Red", "Black", "Blue", "Yellow", "Red"])
print("Label encoded with Yellow:", encoded2)
# Black=0, Blue=1, Red=2, Yellow=3

In [ ]:
# Create the DataFrame used in this part
df = pd.DataFrame({
    'Colour':    ['Red', 'Black', 'Blue'],
    'Fuel Type': ['Petrol', 'Diesel', 'Diesel'],
    'Body':      ['Sedan', 'Sedan', 'Hatchback']
})
print(df)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Fit OneHotEncoder on the 'Colour' column
ohe = preprocessing.OneHotEncoder()
ohe.fit(df[['Colour']])
print("Unique colour categories:", ohe.categories_)

In [ ]:
# Transform — produces a sparse matrix; .toarray() makes it dense
ohe_result = ohe.transform(df[['Colour']]).toarray()
print("One-hot encoded colours:")
print(ohe_result)
# Columns order: Black, Blue, Red
# Row 0 (Red)  → [0, 0, 1]
# Row 1 (Black)→ [1, 0, 0]
# Row 2 (Blue) → [0, 1, 0]

---
###  Exercise 2 — Label encoding vs One-hot encoding

**Task:** Apply both encodings to the `Fuel Type` column, then explain when one-hot is preferred.

In [ ]:
# Work on a copy to avoid mutating the original df
df_tmp = df.copy()

# --- Part 1: Label encoding for 'Fuel Type' ---
le = preprocessing.LabelEncoder()
df_tmp["FuelType_LE"] = le.fit_transform(df_tmp["Fuel Type"])
print("DataFrame with Label Encoded Fuel Type:")
print(df_tmp)
print("\nEncoding mapping:", dict(zip(le.classes_, le.transform(le.classes_))))
# Diesel=0, Petrol=1

In [ ]:
# --- Part 2: One-hot encoding for 'Fuel Type' using pd.get_dummies ---
fuel_ohe = pd.get_dummies(df_tmp["Fuel Type"], prefix="Fuel")
print("One-hot encoded Fuel Type:")
print(fuel_ohe)

# Combine with original (drop the original 'Fuel Type' column)
combined = pd.concat([df_tmp.drop(columns=["Fuel Type", "FuelType_LE"]), fuel_ohe], axis=1)
print("\nCombined DataFrame:")
print(combined)

In [ ]:
# --- Part 3: Explanation ---
explanation = """
When to prefer One-Hot Encoding over Label Encoding:

1. One-hot encoding should be used when the categorical variable is NOMINAL
   (no natural ordering), such as fuel type (Diesel, Petrol, Electric).
   Label encoding would incorrectly imply Diesel < Petrol, which has no meaning.

2. Most ML algorithms (logistic regression, SVM, neural networks) treat
   integer-encoded labels as numeric distances, leading to biased predictions.
   One-hot avoids this by treating each category as an independent binary feature.

3. However, if there are MANY unique categories (high cardinality), one-hot
   encoding creates very wide sparse matrices, so label encoding or target
   encoding may be more practical in those cases.
"""
print(explanation)